In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
train_clean = [x for x in os.listdir('OOF_preds/') if 'train' in x and 'Clean' in x]
test_clean = [x for x in os.listdir('OOF_preds/') if 'test' in x and 'Clean' in x]

train_ner = [x for x in os.listdir('OOF_preds/') if 'train' in x and 'NER' in x]
test_ner = [x for x in os.listdir('OOF_preds/') if 'test' in x and 'NER' in x]

In [3]:
def transform(x):
    a = 0.165 / 0.37
    b =  (1 - 0.165) / (1 - 0.37)
    xt = a * x / (a * x + b * (1 - x))
    return xt

def load_preds(files_list):
    df_preds = pd.DataFrame()
    for i in files_list:
        df_pred = pd.read_pickle('OOF_preds/' + i)
        df_preds = pd.concat([df_preds, df_pred])
    df_preds.sort_index(inplace = True)
    return df_preds

def load_preds_test(files_list, name):
    df_feat1 = pd.DataFrame()
    df_feat2 = pd.DataFrame()
    for i in files_list:
        df_pred = pd.read_pickle('OOF_preds/' + i)
        df_pred.iloc[:, 0] = df_pred.iloc[:, 0].apply(transform)
        df_pred.iloc[:, 1] = df_pred.iloc[:, 1].apply(transform)
        df_feat1 = pd.concat([df_feat1, df_pred.iloc[:, 0]], axis = 1)
        df_feat2 = pd.concat([df_feat2, df_pred.iloc[:, 1]], axis = 1)
    df_feat1.sort_index(inplace = True)
    df_feat2.sort_index(inplace = True)
    df_feat1 = np.mean(df_feat1, axis = 1)
    df_feat2 = np.mean(df_feat2, axis = 1)
    df_preds = pd.concat([df_feat1, df_feat2], axis = 1)
    df_preds.columns = ['attention_feat1_{}'.format(name), 'attention_feat2_{}'.format(name)]
    return df_preds

tr_cl = load_preds(train_clean)
tr_ner = load_preds(train_ner)
te_cl = load_preds_test(test_clean, 'clean')
te_ner = load_preds_test(test_ner, 'ner')

In [ ]:
tr_cl.to_pickle('train_AttentionClean_preds.pkl')
tr_ner.to_pickle('train_AttentionNER_preds.pkl')

In [6]:
te_cl.to_pickle('test_AttentionClean_preds_transformed.pkl')
te_ner.to_pickle('test_AttentionNER_preds_transformed.pkl')